<div style="float: right; width: 70%; height: 200px;">
  <div style="position: relative; top: 20%;padding-right:20%;">
  <div style="font-size:280%;font-weight: bold;padding-top: 20px;"> MARS</div>
  <div style="font-size:150%;font-weight: bold;padding-top: 20px;color: rgba(0 , 0 , 0 , 0.5);"> Plot inSight Data</div>
  </div>
</div>


In [ ]:
import os

import numpy as np
import obspy
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator
from scipy import signal, fft
import pprint as pp

import utils

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 20, 5
plt.rcParams['lines.linewidth'] = 0.5

#%load_ext autoreload
#%autoreload 1
#%matplotlib widget

In [ ]:
## load inSight data to solar
solar = utils.Solar('VF', 'B').get_solar()
#pp.pprint(solar)

trace = solar['S0986c']['VEL']
traceE = trace[0].copy()
traceN = trace[1].copy()
traceZ = trace[2].copy()

data = traceZ.data
npts = traceZ.stats.npts
delta = traceZ.stats.delta
fs = traceZ.stats.sampling_rate
xticks = np.arange(0, npts*delta, delta)

## 自相关

In [ ]:
## 互相关
ZZ = np.correlate(traceZ.data, traceZ.data, 'same')
NZ = np.correlate(traceN.data, traceZ.data, 'same')
EN = np.correlate(traceE.data, traceN.data, 'same')
ZE = np.correlate(traceZ.data, traceE.data, 'same')

fig = plt.figure()
plt.plot(xticks, traceZ)
plt.xlabel('Time [s]'); plt.title('traceN raw signal'); plt.show()
plt.plot(xticks, ZZ)
plt.xlabel('Time [s]'); plt.title('traceZ autocorrelation'); plt.show()
plt.plot(xticks, NZ)
plt.xlabel('Time [s]'); plt.title('traceN and traceZ autocorrelation'); plt.show()
plt.plot(xticks, EN)
plt.xlabel('Time [s]'); plt.title('traceE and traceN autocorrelation'); plt.show()
plt.plot(xticks, ZE)
plt.xlabel('Time [s]'); plt.title('traceZ and traceE autocorrelation'); plt.show()

## H/V

In [ ]:
import scipy.io as io
io.savemat('data.mat', {"yf_E": yf['E'],
                        "yf_N": yf['N'],
                        "yf_Z": yf['Z'],
                        "xf": xf})

In [ ]:
## H/V
yf = {}
xf = fft.fftfreq(npts, delta)
yf['E'] = fft.fft(traceE)
yf['N'] = fft.fft(traceN)
yf['Z'] = fft.fft(traceZ)

def hv2(fE, fN, fZ):
    return np.square((fE**2 + fN**2) / fZ**2)

def hv1(fE, fN, fZ):
    return ((fE + fN)/2.) / fZ

y = hv2(yf['E'].real, yf['N'].real, yf['Z'].real)
y_smooth = signal.savgol_filter(y,53,3)

#yy = y[0:2::]

#plt.plot(yf['E'][0: npts//2])
plt.plot(xf[:npts//2], yf['E'][0: npts//2])
plt.ylim(-100, 100)
plt.xlabel('Frequence [Hz]')
plt.show()

## specgram

In [ ]:
## plot
utils.plot_time(xticks, data)
utils.specgram(data, fs)


def specgram1(data, fs):
    ## plt specgram
    Pxx, freqs, bins, im = plt.specgram(data, Fs=fs)
    plt.xlabel('Time [s]')
    plt.ylabel('Frequence [Hz]')
    plt.xlim(0, 400)
    plt.show()
    
specgram1(data, fs)

#utils.plot_freq(data, delta)

In [ ]:
## 滤波
copy_traceZ = traceZ.copy()
copy_traceZ.filter('bandpass', freqmin=0.1, freqmax=8)

utils.plot_freq( copy_traceZ.data, delta)
utils.plot_time(xticks, copy_traceZ.data)
utils.specgram(copy_traceZ.data, fs)